# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [119]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jan.krueger\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jan.krueger\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [85]:
# load data from database
engine = create_engine('sqlite:///data/messages.db')
df = pd.read_sql_table('messages', engine)
X = df['message'] 
Y = df.drop(columns=['id', 'message', 'original', 'genre'])
with pd.option_context('display.max_colwidth', 480):
    display(X.sample(20))
display(Y)

15644                                                                                                                                                                                          In Guinea a small outbreak at the sub-district level was reported in Siguiri district, predominantly due to Neisseria meningitidis serogroup W135 (Nm W135).
2382                                                                                                                                                                                                                                                                                                  I would like to know what is happening in the country
3914                                                                                                                                                                                                                                                                                                 Please what

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
Y.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.063778,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.244361,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 2. Write a tokenization function to process your text data

In [88]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalnum() and word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


In [91]:
# test tokenize
tokenize('In Guinea a small outbreak at the sub-district level was reported in Siguiri district, predominantly due to Neisseria meningitidis serogroup W135 (Nm W135).')

['in',
 'guinea',
 'small',
 'outbreak',
 'level',
 'reported',
 'siguiri',
 'district',
 'predominantly',
 'due',
 'neisseria',
 'meningitidis',
 'serogroup',
 'w135',
 'nm',
 'w135']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [92]:
pipeline = Pipeline(
        [
            ('vectorize', CountVectorizer(tokenizer=tokenize)),
            ('tfidtransform', TfidfTransformer()),
            ('classify', MultiOutputClassifier(RandomForestClassifier(), n_jobs=1))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [110]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.9)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((2621,), (23595,), (2621, 35), (23595, 35))

In [94]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vectorize',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002369BE8E310>)),
                ('tfidtransform', TfidfTransformer()),
                ('classify',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [95]:
Y_test_pred = pipeline.predict(X_test)

In [96]:
Y_test_pred = pd.DataFrame(Y_test_pred, columns=Y_test.columns)

In [97]:
Y_test_pred.shape, Y_test.shape

((6554, 35), (6554, 35))

In [98]:
Y_test_pred['related'].value_counts(), Y_test['related'].value_counts()

(1    5524
 0     923
 2     107
 Name: related, dtype: int64,
 1    4932
 0    1576
 2      46
 Name: related, dtype: int64)

In [101]:
for c in Y_test_pred.columns:
    print('category: ', c)
    print(classification_report(Y_test[c], Y_test_pred[c], zero_division=0))
    

category:  related
              precision    recall  f1-score   support

           0       0.74      0.43      0.54      1576
           1       0.84      0.94      0.89      4932
           2       0.19      0.43      0.26        46

    accuracy                           0.81      6554
   macro avg       0.59      0.60      0.56      6554
weighted avg       0.81      0.81      0.80      6554

category:  request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5421
           1       0.85      0.49      0.62      1133

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

category:  aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6470
           1       0.00      0.00      0.00        84

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.99      0.98      6554

category:  other_infrastructure
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6248
           1       0.00      0.00      0.00       306

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       

### 6. Improve your model
Use grid search to find better parameters. 

In [103]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorize',
   CountVectorizer(tokenizer=<function tokenize at 0x000002369BE8E310>)),
  ('tfidtransform', TfidfTransformer()),
  ('classify',
   MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=1))],
 'verbose': False,
 'vectorize': CountVectorizer(tokenizer=<function tokenize at 0x000002369BE8E310>),
 'tfidtransform': TfidfTransformer(),
 'classify': MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=1),
 'vectorize__analyzer': 'word',
 'vectorize__binary': False,
 'vectorize__decode_error': 'strict',
 'vectorize__dtype': numpy.int64,
 'vectorize__encoding': 'utf-8',
 'vectorize__input': 'content',
 'vectorize__lowercase': True,
 'vectorize__max_df': 1.0,
 'vectorize__max_features': None,
 'vectorize__min_df': 1,
 'vectorize__ngram_range': (1, 1),
 'vectorize__preprocessor': None,
 'vectorize__stop_words': None,
 'vectorize__strip_accents': None,
 'vectorize__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vectorize__tokenizer': <func

In [112]:

parameters = [
    {'classify__estimator': [RandomForestClassifier()], 'classify__estimator__n_estimators': [20, 50]},
    {'classify__estimator': [AdaBoostClassifier()], 'classify__estimator__n_estimators': [20, 50]},
    {'classify__estimator': [MultinomialNB()]}
]

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [113]:
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vectorize',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002369BE8E310>)),
                                       ('tfidtransform', TfidfTransformer()),
                                       ('classify',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(),
                                                              n_jobs=1))]),
             param_grid=[{'classify__estimator': [RandomForestClassifier(n_estimators=20)],
                          'classify__estimator__n_estimators': [20, 50]},
                         {'classify__estimator': [AdaBoostClassifier()],
                          'classify__estimator__n_estimators': [20, 50]},
                         {'classify__estimator': [MultinomialNB()]}])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [116]:
pd.DataFrame(cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classify__estimator,param_classify__estimator__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,39.248379,2.342895,8.936005,0.848832,RandomForestClassifier(n_estimators=20),20,{'classify__estimator': RandomForestClassifier...,0.186667,0.185115,0.217557,0.215649,0.208015,0.202601,0.014021,1
1,48.927163,3.352130,10.294797,1.747777,RandomForestClassifier(n_estimators=20),50,{'classify__estimator': RandomForestClassifier...,0.175238,0.162214,0.211832,0.232824,0.229008,0.202223,0.028549,2
2,41.404684,3.179950,8.548497,0.262188,AdaBoostClassifier(),20,"{'classify__estimator': AdaBoostClassifier(), ...",0.196190,0.185115,0.185115,0.209924,0.137405,0.182750,0.024439,3
3,56.218647,2.050483,10.548169,0.589603,AdaBoostClassifier(),50,"{'classify__estimator': AdaBoostClassifier(), ...",0.188571,0.173664,0.173664,0.179389,0.162214,0.175501,0.008594,4
4,32.193466,1.824663,7.483489,0.552145,MultinomialNB(),NaN,{'classify__estimator': MultinomialNB()},0.123810,0.133588,0.166031,0.135496,0.141221,0.140029,0.014161,5


### 9. Export your model as a pickle file

In [120]:
file_name = './data/model.pickle'
with open (file_name, 'wb') as f:
    pickle.dump(cv.best_estimator_, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.